# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [1]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

ws = Workspace.from_config()

experiment_name = 'Automlrun'

experiment=Experiment(ws, experiment_name)

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

compute_cluster_name = 'p12'

try:
    compute_target = ComputeTarget(workspace=ws, name = compute_cluster_name)
    print('Found the compute cluster')

except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D3_V2", max_nodes=4)
    compute_target = ComputeTarget.create(ws, compute_cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True)

Found the compute cluster


In [3]:
from azureml.core.dataset import Dataset
from sklearn.model_selection import train_test_split

key = 'heart_failure_clinical_records_dataset'
if key in ws.datasets.keys():
    dataset = ws.datasets[key]
    print("dataset found!")

else:
    url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv'
    dataset = Dataset.Tabular.from_delimited_files(url)
    dataset = dataset.register(ws,key)

df = dataset.to_pandas_dataframe()
train, test = train_test_split(df, shuffle=True)
train.to_csv('train.csv',index = False)

datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./train.csv'])

train = Dataset.Tabular.from_delimited_files([(datastore,'train.csv')])

dataset found!
Uploading an estimated of 1 files
Target already exists. Skipping upload for train.csv
Uploaded 0 files


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [4]:
from azureml.train.automl import AutoMLConfig

# TODO: Put your automl settings here
automl_settings = {"primary_metric":"accuracy", "experiment_timeout_minutes":30, "enable_early_stopping":True, "n_cross_validations":3,"max_concurrent_iterations": 4}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target = compute_target, task = 'classification', training_data = train, label_column_name = 'DEATH_EVENT',blocked_models=['XGBoostClassifier'],**automl_settings)

In [5]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on p12 with default configuration
Running on remote compute: p12
Parent Run ID: AutoML_d22489fc-a63f-41bb-9d47-f59ae655801d

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/A

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
from azureml.widgets import RunDetails

RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_d22489fc-a63f-41bb-9d47-f59ae655801d',
 'target': 'p12',
 'status': 'Completed',
 'startTimeUtc': '2021-04-04T06:18:00.697983Z',
 'endTimeUtc': '2021-04-04T06:54:15.748606Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'p12',
  'AMLSettingsJsonString': '{"path":null,"name":"Automlrun","subscription_id":"48a74bb7-9950-4cc1-9caa-5d50f995cc55","resource_group":"aml-quickstarts-141917","workspace_name":"quick-starts-ws-141917","region":"southcentralus","compute_target":"p12","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":null,"num_classes":null,"feat

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [7]:
!pip install xgboost==0.90

     |████████████████████████████████| 142.8 MB 28 kB/s s eta 0:00:01
  Attempting uninstall: xgboost
    Found existing installation: xgboost 1.3.3
    Uninstalling xgboost-1.3.3:
      Successfully uninstalled xgboost-1.3.3


In [8]:
best_run, model = remote_run.get_output()
print(best_run)
print(model)
best_run.get_tags()
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Run(Experiment: Automlrun,
Id: AutoML_d22489fc-a63f-41bb-9d47-f59ae655801d_34,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                reg_alpha=0.3157894736842105,
                                                                                                reg_lambda=0.05263157894736842,
               

In [9]:
best_run_metrics

{'accuracy': 0.866126126126126,
 'precision_score_micro': 0.866126126126126,
 'average_precision_score_weighted': 0.9175516663682496,
 'balanced_accuracy': 0.8213972861371008,
 'recall_score_macro': 0.8213972861371008,
 'f1_score_weighted': 0.8613891083804335,
 'recall_score_micro': 0.866126126126126,
 'precision_score_weighted': 0.8730415574157968,
 'norm_macro_recall': 0.6427945722742017,
 'log_loss': 0.39072920461304617,
 'weighted_accuracy': 0.9001413777044823,
 'AUC_micro': 0.9106636690744798,
 'matthews_correlation': 0.6840897998034564,
 'average_precision_score_micro': 0.9144514847971504,
 'precision_score_macro': 0.8654471523893431,
 'average_precision_score_macro': 0.888046372465075,
 'recall_score_weighted': 0.866126126126126,
 'AUC_weighted': 0.9103752541500257,
 'f1_score_micro': 0.866126126126126,
 'AUC_macro': 0.9103752541500256,
 'f1_score_macro': 0.833098420599657,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_d22489fc-a63f-41bb-9d47-f59ae655801d_34/c